# Ejercicios Pair Programming - Módulo 2 Sprint 2
## Lección 2 - ETL 2.1
### Natalia García Lago y Noemi Azorín

In [1]:
import requests
import pandas as pd
import numpy as np
import ast

pd.options.display.max_columns = None
from datetime import datetime, timedelta

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.

In [2]:
df_ataques = pd.read_csv("../ficheros/attacks_limpieza_completa.csv", index_col = 0)
df_ataques.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925
4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.658512,-102.007710,HN,1,0,0,1,0,0,2,7,4,6.313925


En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1 - El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

2 - El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

In [3]:
df_info = pd.read_csv("../ficheros/ETL1.csv", index_col= 0)
df_info.reset_index()

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,1,0,335,2,Papua New Guinea
1,1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",16,-1,13,-9999,rain,1,0,10,2,Papua New Guinea
2,2,9,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",18,-4,12,-9999,rain,3,0,40,2,Papua New Guinea
3,3,12,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",18,-4,12,-9999,rain,3,0,25,2,Papua New Guinea
4,4,15,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,3,0,340,2,Papua New Guinea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,315,180,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",0,10,15,1008,snow,3,4,85,2,USA
316,316,183,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",0,10,16,1008,snow,3,5,50,2,USA
317,317,186,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",0,10,15,1008,snow,4,5,10,2,USA
318,318,189,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",0,10,15,1009,snow,4,5,0,2,USA


El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos.

- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:

    USA,
    Australia,
    New Zealand,
    South Africa,
    Papua New Guinea

- Del dataframe de los datos climáticos seleccionaremos todas las columnas.

- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

- Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

    - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [4]:
lista_selec = ['usa', 'australia', 'papua new guinea', 'new zealand', "south africa"]

In [5]:
df_ataques_paises = df_ataques[df_ataques["country"].isin(lista_selec)]

In [6]:
#comprobamos que se ha generado un nuevo df con los paises filtrados
df_ataques_paises["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [7]:
df_info['wind_profile'] = df_info['wind_profile'].apply(ast.literal_eval)

In [8]:
df_info['rh_profile'] = df_info['rh_profile'].apply(ast.literal_eval)

In [9]:
y = df_info['wind_profile'].apply(pd.Series)

In [10]:
x = df_info['rh_profile'].apply(pd.Series)

In [11]:
df_info.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,1,0,335,2,Papua New Guinea
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",16,-1,13,-9999,rain,1,0,10,2,Papua New Guinea


In [12]:
for i in range(len(x.columns)):
    nombre_rh = "rh_"+ str(x[i].apply(pd.Series)["layer"][0])
    valores_rh = (x[i].apply(pd.Series)["rh"])

    df_info.insert(i, nombre_rh, valores_rh)

In [13]:
df_info.head()

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,-9999,-9999,12,12,13,13,14,14,13,8,10,13,14,15,14,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,1,0,335,2,Papua New Guinea
1,-9999,-9999,-9999,11,12,13,13,14,14,13,13,13,14,15,15,12,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",16,-1,13,-9999,rain,1,0,10,2,Papua New Guinea
2,-9999,-9999,-9999,13,14,12,13,14,15,13,11,13,14,15,16,15,9,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",18,-4,12,-9999,rain,3,0,40,2,Papua New Guinea
3,-9999,-9999,-9999,13,14,10,11,13,12,11,11,10,7,14,14,4,12,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",18,-4,12,-9999,rain,3,0,25,2,Papua New Guinea
4,-9999,-9999,-9999,13,13,10,10,10,10,10,9,4,7,16,16,5,15,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,3,0,340,2,Papua New Guinea


In [14]:
for i in range(len(y.columns)):
    nombre_wind = "wind_"+ str(y[i].apply(pd.Series)["layer"][0])
    valores_wind = list(y[i].apply(pd.Series)["direction"])
    
    df_info.insert(i, nombre_wind, valores_wind)

In [15]:
df_info.head()

,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,-9999,-9999,305,345,355,15,50,90,100,90,85,115,100,110,100,-9999,-9999,-9999,12,12,13,13,14,14,13,8,10,13,14,15,14,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,1,0,335,2,Papua New Guinea
1,-9999,-9999,-9999,10,350,350,5,60,85,90,85,90,80,100,105,105,-9999,-9999,-9999,11,12,13,13,14,14,13,13,13,14,15,15,12,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",16,-1,13,-9999,rain,1,0,10,2,Papua New Guinea
2,-9999,-9999,-9999,20,355,345,355,35,65,85,85,90,95,110,110,105,-9999,-9999,-9999,13,14,12,13,14,15,13,11,13,14,15,16,15,9,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",18,-4,12,-9999,rain,3,0,40,2,Papua New Guinea
3,-9999,-9999,-9999,10,350,340,10,55,80,85,90,90,70,85,110,110,-9999,-9999,-9999,13,14,10,11,13,12,11,11,10,7,14,14,4,12,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",18,-4,12,-9999,rain,3,0,25,2,Papua New Guinea
4,-9999,-9999,-9999,0,345,345,10,50,70,80,90,85,75,95,125,125,-9999,-9999,-9999,13,13,10,10,10,10,10,9,4,7,16,16,5,15,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,3,0,340,2,Papua New Guinea


In [16]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 49 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   wind_950mb         320 non-null    int64 
 1   wind_900mb         320 non-null    int64 
 2   wind_850mb         320 non-null    int64 
 3   wind_800mb         320 non-null    int64 
 4   wind_750mb         320 non-null    int64 
 5   wind_700mb         320 non-null    int64 
 6   wind_650mb         320 non-null    int64 
 7   wind_600mb         320 non-null    int64 
 8   wind_550mb         320 non-null    int64 
 9   wind_500mb         320 non-null    int64 
 10  wind_450mb         320 non-null    int64 
 11  wind_400mb         320 non-null    int64 
 12  wind_350mb         320 non-null    int64 
 13  wind_300mb         320 non-null    int64 
 14  wind_250mb         320 non-null    int64 
 15  wind_200mb         320 non-null    int64 
 16  rh_950mb           320 non-null    int64 
 1

In [17]:
df_groupby = df_info.groupby("country").mean()

/tmp/ipykernel_693/175728612.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_groupby = df_info.groupby("country").mean()


In [18]:
df_groupby

,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,89.375000,95.703125,109.609375,113.906250,118.906250,123.750000,134.062500,137.421875,151.484375,168.125000,171.406250,218.593750,257.265625,267.578125,271.328125,269.375000,3.015625,4.000000,5.218750,6.828125,9.281250,11.250000,10.671875,7.593750,1.890625,-0.53125,-2.546875,-2.328125,-2.125000,-2.843750,-3.375000,-3.218750,97.5,3.375000,-9999.0,-9999.0,-9999.0,32.500000,-0.234375,2.578125,1005.687500,2.984375,0.0000,106.328125,2.921875
New Zealand,-1783.171875,130.234375,158.906250,172.187500,174.609375,165.156250,152.578125,150.234375,151.015625,166.562500,184.062500,193.593750,205.156250,192.187500,196.718750,202.421875,-1867.359375,9.750000,10.718750,10.375000,8.734375,6.656250,3.609375,1.171875,0.359375,1.65625,2.265625,2.468750,2.375000,3.687500,5.078125,-1.015625,97.5,5.078125,-9999.0,-9999.0,-9999.0,15.140625,6.546875,9.843750,1017.890625,2.984375,0.0000,156.328125,1.984375
Papua New Guinea,-9999.000000,-9999.000000,-9999.000000,167.265625,232.343750,273.828125,249.687500,151.640625,99.921875,108.906250,94.375000,76.640625,80.937500,102.109375,109.062500,110.312500,-9999.000000,-9999.000000,-9999.000000,11.796875,12.406250,11.421875,10.765625,11.109375,11.281250,10.21875,8.734375,8.031250,9.218750,11.453125,10.703125,9.859375,97.5,8.437500,-9999.0,-9999.0,-9999.0,14.984375,-0.046875,13.171875,-9999.000000,4.015625,0.0000,200.156250,2.000000
South Africa,-9999.000000,-9999.000000,148.359375,187.343750,189.140625,195.781250,177.812500,148.906250,139.218750,129.453125,170.156250,225.156250,241.796875,252.187500,286.171875,293.437500,-9999.000000,-9999.000000,-0.718750,-0.156250,0.828125,2.062500,3.906250,6.703125,8.125000,2.75000,-1.531250,-0.765625,0.890625,1.234375,2.031250,4.187500,97.5,3.468750,-9999.0,-9999.0,-9999.0,32.015625,1.843750,-0.734375,1010.093750,0.375000,0.0000,146.250000,3.046875
USA,-9999.000000,176.328125,187.890625,204.140625,225.625000,251.484375,248.046875,246.640625,255.468750,267.265625,262.578125,260.781250,254.921875,254.531250,271.484375,268.437500,-9999.000000,8.171875,6.515625,4.796875,4.984375,5.359375,6.203125,6.968750,7.359375,7.81250,8.125000,8.796875,10.265625,9.546875,4.375000,0.875000,97.5,6.375000,-9999.0,-9999.0,-9999.0,2.828125,13.234375,9.046875,1012.281250,1.078125,-1405.4375,178.750000,2.453125


In [24]:
df_final = pd.concat([df_groupby, df_ataques_paises], axis = 0)

In [25]:
df_final

,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
Australia,89.375000,95.703125,109.609375,113.906250,118.906250,123.750000,134.062500,137.421875,151.484375,168.125000,171.406250,218.593750,257.265625,267.578125,271.328125,269.375000,3.015625,4.000000,5.218750,6.828125,9.281250,11.250000,10.671875,7.593750,1.890625,-0.53125,-2.546875,-2.328125,-2.125000,-2.843750,-3.375000,-3.218750,97.5,3.375000,-9999.0,-9999.0,-9999.0,32.500000,-0.234375,2.578125,1005.687500,2.984375,0.0000,106.328125,2.921875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Zealand,-1783.171875,130.234375,158.906250,172.187500,174.609375,165.156250,152.578125,150.234375,151.015625,166.562500,184.062500,193.593750,205.156250,192.187500,196.718750,202.421875,-1867.359375,9.750000,10.718750,10.375000,8.734375,6.656250,3.609375,1.171875,0.359375,1.65625,2.265625,2.468750,2.375000,3.687500,5.078125,-1.015625,97.5,5.078125,-9999.0,-9999.0,-9999.0,15.140625,6.546875,9.843750,1017.890625,2.984375,0.0000,156.328125,1.984375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Papua New Guinea,-9999.000000,-9999.000000,-9999.000000,167.265625,232.343750,273.828125,249.687500,151.640625,99.921875,108.906250,94.375000,76.640625,80.937500,102.109375,109.062500,110.312500,-9999.000000,-9999.000000,-9999.000000,11.796875,12.406250,11.421875,10.765625,11.109375,11.281250,10.21875,8.734375,8.031250,9.218750,11.453125,10.703125,9.859375,97.5,8.437500,-9999.0,-9999.0,-9999.0,14.984375,-0.046875,13.171875,-9999.000000,4.015625,0.0000,200.156250,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
South Africa,-9999.000000,-9999.000000,148.359375,187.343750,189.140625,195.781250,177.812500,148.906250,139.218750,129.453125,170.156250,225.156250,241.796875,252.187500,286.171875,293.437500,-9999.000000,-9999.000000,-0.718750,-0.156250,0.828125,2.062500,3.906250,6.703125,8.125000,2.75000,-1.531250,-0.765625,0.890625,1.234375,2.031250,4.187500,97.5,3.468750,-9999.0,-9999.0,-9999.0,32.015625,1.843750,-0.734375,1010.093750,0.375000,0.0000,146.250000,3.046875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USA,-9999.000000,176.328125,187.890625,204.140625,225.625000,251.484375,248.046875,246.640625,255.468750,267.265625,262.578125,260.781250,254.921875,254.531250,271.484375,268.437500,-9999.000000,8.171875,6.515625,4.796875,4.984375,5.359375,6.203125,6.968750,7.359375,7.81250,8.125000,8.796875,10.265625,9.546875,4.375000,0.875000,97.5,6.375000,-9999.0,-9999.0,-9999.0,2.828125,13.234375,9.046875,1012.281250,1.078125,-1405.4375,178.750000,2.453125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Unprovoked,south africa,27.0,Unespecific,Jul,Y,M,-28.816624,24.991639,HS,0.0,0.0,1.0,0.0,0.0,1.0,3.0,6.0,7.0,6.313925
6212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [28]:
df_final.to_csv("../ficheros/ETL2.csv")